In [13]:
import cv2 as cv
import numpy as np
import os

In [14]:
def show_image(title,image):
    image=cv.resize(image,(0,0),fx=0.3,fy=0.3)
    cv.imshow(title,image)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [15]:
# show_image("image", cv.imread("../Proiect1/CAVA-2024-Tema1/antrenare/1_01.jpg"))

In [16]:
def select_board_position_and_border_size(image):
    image_original = image.copy()
    image = cv.cvtColor(image,cv.COLOR_BGR2HSV)
    image_m_blur = cv.medianBlur(image,3)
    image_g_blur = cv.GaussianBlur(image_m_blur, (0, 0), 5) 
    image_sharpened = cv.addWeighted(image_m_blur, 1.2, image_g_blur, -0.8, 0)
    
    masca_initiala = np.array([50, 0, 0])
    masca_finala = np.array([255, 255, 255])
    combined_mask = cv.inRange(image_g_blur, masca_initiala, masca_finala)
    # # show_image('combined_mask',combined_mask)
    
    contours, _ = cv.findContours(combined_mask,  cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    
    max_area = 0
    for i in range(len(contours)):
        if(len(contours[i]) > 3):
            possible_top_left = None
            possible_bottom_right = None
            for point in contours[i].squeeze():
                if possible_top_left is None or point[0] + point[1] < possible_top_left[0] + possible_top_left[1]:
                    possible_top_left = point

                if possible_bottom_right is None or point[0] + point[1] > possible_bottom_right[0] + possible_bottom_right[1] :
                    possible_bottom_right = point

            diff = np.diff(contours[i].squeeze(), axis = 1)
            possible_top_right = contours[i].squeeze()[np.argmin(diff)]
            possible_bottom_left = contours[i].squeeze()[np.argmax(diff)]
            if cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]])) > max_area:
                max_area = cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]]))
                top_left = possible_top_left
                bottom_right = possible_bottom_right
                top_right = possible_top_right
                bottom_left = possible_bottom_left
                
    width = 2000
    height = 2000
    
    image_copy = image.copy()
    cv.circle(image_copy,tuple(top_left),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(top_right),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_left),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_right),20,(0,0,255),-1)
    
    puzzle = np.array([top_left,top_right,bottom_right,bottom_left], dtype = "float32")
    destination_of_puzzle = np.array([[0,0],[width,0],[width,height],[0,height]], dtype = "float32")

    M = cv.getPerspectiveTransform(puzzle,destination_of_puzzle)

    result = cv.warpPerspective(image, M, (width, height))
    # # show_image("result",result)
    
    return result, cv.warpPerspective(image_original, M, (width, height))

In [17]:
def gaseste_colturile_celui_mai_mare_contur(imagine):
    contours, _ = cv.findContours(imagine,  cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    ######### # show_image('combined_mask',combined_mask)
    # # show_image('image_thresholded',thresh)
    
    max_area = 0
    for i in range(len(contours)):
        if(len(contours[i]) > 3):
            possible_top_left = None
            possible_bottom_right = None
            for point in contours[i].squeeze():
                if possible_top_left is None or point[0] + point[1] < possible_top_left[0] + possible_top_left[1]:
                    possible_top_left = point

                if possible_bottom_right is None or point[0] + point[1] > possible_bottom_right[0] + possible_bottom_right[1] :
                    possible_bottom_right = point

            diff = np.diff(contours[i].squeeze(), axis = 1)
            possible_top_right = contours[i].squeeze()[np.argmin(diff)]
            possible_bottom_left = contours[i].squeeze()[np.argmax(diff)]
            if cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]])) > max_area:
                max_area = cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]]))
                top_left = possible_top_left
                bottom_right = possible_bottom_right
                top_right = possible_top_right
                bottom_left = possible_bottom_left
    return top_left, top_right, bottom_right, bottom_left

In [18]:
def select_inner_grid_position_and_size_in_board(board_image_hsv, board_image_original):
    # show_image('board_image_original',board_image_original)
    # show_image('board_image_hsv',board_image_hsv)
    original_image_copy = board_image_original.copy()
    
    # Save image to local
    cv.imwrite('board_image_original.jpg', board_image_original)
    cv.imwrite('board_image_hsv.jpg', board_image_hsv)
    
    
    
    # Ducem originala in HSV
    board_image_original = cv.cvtColor(board_image_original, cv.COLOR_BGR2HSV)
    # Apply thresholding
    masca_initiala = np.array([0, 0, 135])
    masca_finala = np.array([100, 255, 255])
    combined_mask = cv.inRange(board_image_original, masca_initiala, masca_finala)
    # show_image('combined_mask',combined_mask)
    
    
    ##### Aici trebuie sa detectam colturile imaginii
    # Prima data, selectam pozitiile aproximative ale colturilor imaginii, ca dreptunghiuri, apoi detectam colturile exacte in acele contururi
    colt_posibil_dreapta_sus = [(1680, 190), (1790, 320)]
    colt_posibil_dreapta_jos = [(1680, 1690), (1770, 1790)]
    # colt_posibil_stanga_sus = [(190, 190), (320, 350)]
    colt_posibil_stanga_jos = [(200, 1680), (310, 1790)]
    # Desenam contururile
    cv.rectangle(combined_mask, colt_posibil_dreapta_sus[0], colt_posibil_dreapta_sus[1], (0, 0, 255), 2)
    cv.rectangle(combined_mask, colt_posibil_dreapta_jos[0], colt_posibil_dreapta_jos[1], (0, 0, 255), 2)
    cv.rectangle(combined_mask, colt_posibil_stanga_jos[0], colt_posibil_stanga_jos[1], (0, 0, 255), 2)
    # cv.rectangle(combined_mask, colt_posibil_stanga_sus[0], colt_posibil_stanga_sus[1], (0, 0, 255), 2)
    # show_image('combined_mask',combined_mask)
    # Taiem imaginea in functie de colturile detectate
    dreapta_sus = combined_mask[colt_posibil_dreapta_sus[0][1]:colt_posibil_dreapta_sus[1][1], colt_posibil_dreapta_sus[0][0]:colt_posibil_dreapta_sus[1][0]]
    dreapta_jos = combined_mask[colt_posibil_dreapta_jos[0][1]:colt_posibil_dreapta_jos[1][1], colt_posibil_dreapta_jos[0][0]:colt_posibil_dreapta_jos[1][0]]
    # stanga_sus = combined_mask[colt_posibil_stanga_sus[0][1]:colt_posibil_stanga_sus[1][1], colt_posibil_stanga_sus[0][0]:colt_posibil_stanga_sus[1][0]]
    stanga_jos = combined_mask[colt_posibil_stanga_jos[0][1]:colt_posibil_stanga_jos[1][1], colt_posibil_stanga_jos[0][0]:colt_posibil_stanga_jos[1][0]]
    
    # Detectam colturile exacte
    top_left, top_right, bottom_right, bottom_left = gaseste_colturile_celui_mai_mare_contur(dreapta_sus)
    top_left_dreapta_jos, top_right_dreapta_jos, bottom_right_dreapta_jos, bottom_left_dreapta_jos = gaseste_colturile_celui_mai_mare_contur(dreapta_jos)
    # top_left_stanga_sus, top_right_stanga_sus, bottom_right_stanga_sus, bottom_left_stanga_sus = gaseste_colturile_celui_mai_mare_contur(stanga_sus)
    top_left_stanga_jos, top_right_stanga_jos, bottom_right_stanga_jos, bottom_left_stanga_jos = gaseste_colturile_celui_mai_mare_contur(stanga_jos)
    
    nou_punct_topright = tuple(top_right + np.array(colt_posibil_dreapta_sus[0]))
    nou_punct_bottomright = tuple(bottom_right_dreapta_jos + np.array(colt_posibil_dreapta_jos[0]))
    nou_punct_bottomleft = tuple(bottom_left_stanga_jos + np.array(colt_posibil_stanga_jos[0]))
    # Bazat pe cele 3 colturi, calculam al 4-lea colt
    nou_punct_topleft = nou_punct_topright[1], nou_punct_bottomleft[0]
    
    # Desenam cercuri in colturile detectate
    # Pentru dreapta sus, adaugam coltul top_right al dreapta sus si desenam un cerc rosu
    cv.circle(board_image_original,nou_punct_topright,20,(0,0,255),-1)
    # Pentru dreapta jos, adaugam coltul top_right al dreapta jos si desenam un cerc rosu
    cv.circle(board_image_original,nou_punct_bottomright,20,(0,0,255),-1)
    # Pentru stanga jos, adaugam coltul top_right al stanga jos si desenam un cerc rosu
    cv.circle(board_image_original,nou_punct_bottomleft,20,(0,0,255),-1)
    # Desenam cercul pentru coltul stanga sus
    cv.circle(board_image_original,nou_punct_topleft,20,(0,0,255),-1)
    # show_image('board_image_original',board_image_original)
    
    # Transformam imaginea in perspectiva
    width = 2000
    height = 2000
    puzzle = np.array([nou_punct_topleft,nou_punct_topright,nou_punct_bottomright,nou_punct_bottomleft], dtype = "float32")
    destination_of_puzzle = np.array([[0,0],[width,0],[width,height],[0,height]], dtype = "float32")
    M = cv.getPerspectiveTransform(puzzle,destination_of_puzzle)
    result = cv.warpPerspective(original_image_copy, M, (width, height))
    
    # Draw the separating grid lines
    square_width = width // 14 + 1
    square_height = height // 14 + 1
    print('square_width',square_width)
    print('square_height',square_height)
    # Draw the grid
    grid_line_thickness = 5
    # Grid line color is black
    for i in range(14):
        cv.line(result, (i * square_width, 0), (i * square_width, height), (0, 0, 0), grid_line_thickness)
        cv.line(result, (0, i * square_height), (width, i * square_height), (0, 0, 0), grid_line_thickness)
        
    # Impartim imaginea in 14x14
    width, height = combined_mask.shape[:2]
    square_width, square_height = width // 14 + 1, height // 14 + 1
    copie_imagine = result.copy()
    copie_imagine = cv.cvtColor(copie_imagine, cv.COLOR_BGR2HSV)
    masca_imagine_detectie_piese_lipsa_centru_lower = np.array([0, 100, 0])
    masca_imagine_detectie_piese_lipsa_centru_upper = np.array([35, 255, 255])
    masca_imagine_detectie_piese_lipsa_centru = cv.inRange(copie_imagine, masca_imagine_detectie_piese_lipsa_centru_lower, masca_imagine_detectie_piese_lipsa_centru_upper)
    for i in range(14):
        for j in range(14):
            square = masca_imagine_detectie_piese_lipsa_centru[i * square_width:(i + 1) * square_width, j * square_height:(j + 1) * square_height]
            new_square = masca_imagine_detectie_piese_lipsa_centru[i * square_width + 20:(i + 1) * square_width - 20, j * square_height + 20:(j + 1) * square_height - 20]
            if square.size > 0:
                colors, counts = np.unique(square, return_counts=True)
                majoritar = colors[np.argmax(counts)]
                if majoritar == 255:
                    # Make the current square in the result pitch black
                    result[i * square_width:(i + 1) * square_width, j * square_height:(j + 1) * square_height] = 0
    # show_image('result_after_modified_colors',result)
    return result
    
    
    # # print(top_left,top_right,bottom_left,bottom_right)
    # # Extragem imaginea cu tabla de joc
    # width = 2000
    # height = 2000
    
    # image_copy = board_image_original.copy()
    # cv.circle(image_copy,tuple(top_left),20,(0,0,255),-1)
    # cv.circle(image_copy,tuple(top_right),20,(0,0,255),-1)
    # cv.circle(image_copy,tuple(bottom_left),20,(0,0,255),-1)
    # cv.circle(image_copy,tuple(bottom_right),20,(0,0,255),-1)
    # # show_image("detected corners",image_copy)
    
    # puzzle = np.array([top_left,top_right,bottom_right,bottom_left], dtype = "float32")
    # destination_of_puzzle = np.array([[0,0],[width,0],[width,height],[0,height]], dtype = "float32")

    # M = cv.getPerspectiveTransform(puzzle,destination_of_puzzle)

    # result = cv.warpPerspective(board_image_original, M, (width, height))
    # # Divide the board into 14x14 squares
    # square_width = width // 14
    # square_height = height // 14
    # print('square_width',square_width)
    # print('square_height',square_height)
    # # Draw the grid
    # grid_line_thickness = 8
    # # Grid line color is black
    # for i in range(14):
    #     cv.line(result, (i * square_width, 0), (i * square_width, height), (0, 0, 0), grid_line_thickness)
    #     cv.line(result, (0, i * square_height), (width, i * square_height), (0, 0, 0), grid_line_thickness)
    
    # # Write image to local
    # cv.imwrite('resulting_board_item.jpg', result)
    # # show_image("result",result)
    # return result

In [19]:
select_board_position_and_border_size(cv.imread("../Proiect1/CAVA-2024-Tema1/imagini_auxiliare/04.jpg"))

(array([[[  2,  83, 110],
         [  2,  74, 110],
         [  0,  72, 106],
         ...,
         [178, 103,  89],
         [  1, 114,  94],
         [  3, 123,  91]],
 
        [[175,  60, 107],
         [165,  54, 102],
         [162,  54,  96],
         ...,
         [172,  90,  88],
         [177,  99,  90],
         [  1, 109,  91]],
 
        [[153,  38,  96],
         [138,  43, 101],
         [129,  50, 103],
         ...,
         [160,  77,  80],
         [170,  91,  84],
         [177,  99,  90]],
 
        ...,
 
        [[174,  84, 100],
         [164,  67,  93],
         [144,  54,  86],
         ...,
         [144,  49,  73],
         [157,  47,  70],
         [161,  57,  72]],
 
        [[172,  92, 100],
         [172,  80,  97],
         [163,  62,  85],
         ...,
         [162,  59,  74],
         [166,  60,  72],
         [173,  71,  75]],
 
        [[  0, 103,  99],
         [166,  90, 100],
         [172,  73,  95],
         ...,
         [172,  74,  79],
  

In [20]:
def altereaza_imaginea_cu_grid_ul(image):
    # show_image('imagine_cu_careul_de_procesat',image)
    # Write image
    cv.imwrite('imagine_cu_careul_de_procesat2.jpg', image)
    
    image = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    masca_pentru_a_vedea_pozitiile_cifrelor_inf = np.array([0, 0, 95])
    masca_pentru_a_vedea_pozitiile_cifrelor_sup = np.array([255, 255, 255])
    # Aplicam masca pentru a vedea pozitiile cifrelor
    combined_mask = cv.inRange(image, masca_pentru_a_vedea_pozitiile_cifrelor_inf, masca_pentru_a_vedea_pozitiile_cifrelor_sup)
    # show_image('imagine_cu_careul_de_procesat',combined_mask)
    
    pozitii_deja_utilizate_chr = []
    pozitii_deja_utilizate_tuple = []
    
    # Impartim imaginea in 14x14
    width, height = combined_mask.shape[:2]
    square_width, square_height = width // 14 + 1, height // 14 + 1
    # Pentru fiecare patrat, calculam media intensitatii
    for i in range(14):
        for j in range(14):
            # Calculam media intensitatii
            square = combined_mask[i * square_width:(i + 1) * square_width, j * square_height:(j + 1) * square_height]
            # Create a new square that is distance 20 away from the original square, inside the original square
            new_square = combined_mask[i * square_width + 20:(i + 1) * square_width - 20, j * square_height + 20:(j + 1) * square_height - 20]
            # Draw the new square referring to its shape
            cv.rectangle(combined_mask, (j * square_height + 20, i * square_width + 20), ((j + 1) * square_height - 20, (i + 1) * square_width - 20), (0, 255, 0), 2)
            
            # Calculam media intensitatii
            # print(np.mean(new_square))
            mean_intensity = np.mean(new_square)
            # Daca media intensitatii este mai mica decat 100, atunci este un patrat negru
            if mean_intensity < 240 and mean_intensity > 0: # Era 235 inainte
                # Avem un patrat plin, il coloram cu grey
                combined_mask[i * square_width:(i + 1) * square_width, j * square_height:(j + 1) * square_height] = 100
                pozitii_deja_utilizate_chr.append(str(i + 1) + str(chr(ord('A') + j)))
                pozitii_deja_utilizate_tuple.append((i, j))
                pass
    # show_image('imagine_cu_careul_de_procesat',combined_mask)
    return square_width, square_height, pozitii_deja_utilizate_chr, pozitii_deja_utilizate_tuple
    

In [21]:
#########################
#########################
# For the digit classification

def alter_square(square):
    square_height, square_width = square.shape
    
    # We remove the eventual white noise from the square
    _, threshold = cv.threshold(square, 127, 255, cv.THRESH_BINARY)
    # Perform a morphological closing operation to remove the noise
    kernel = np.ones((3, 3), np.uint8)
    square = cv.morphologyEx(threshold, cv.MORPH_CLOSE, kernel, iterations=2)
    
    # show_image('altered_square_after_morphological', square)
    # Reduce suquare size to height-10 x width-10
    square = square[10:square_height - 10, 10:square_width - 10]
    # Show the new altered square
    # show_image('altered_square_after_cut', square)
    
    white_pixels = np.where(square == 255)
    min_x, min_y = np.min(white_pixels, axis=1)
    max_x, max_y = np.max(white_pixels, axis=1)
    
    # Crop the square to the bounding box
    square = square[min_x:max_x, min_y:max_y]
    # Show the square
    # show_image('square', square)
    # Resize the square to 100x100 by adding black pixels and keeping the original rectangle centered
    new_square = np.zeros((150, 150), dtype=np.uint8)
    new_square_height, new_square_width = new_square.shape
    # Calculate the offset
    offset_x = (new_square_width - square.shape[1]) // 2
    offset_y = (new_square_height - square.shape[0]) // 2
    # Copy the square into the new square
    new_square[offset_y:offset_y + square.shape[0], offset_x:offset_x + square.shape[1]] = square
    
    return new_square
    
def create_dataset_from_given_square(square, number_inside_square):
    # Create output directory if it doesn't exist
    output_dir = f'dataset/digit_{number_inside_square}'
    os.makedirs(output_dir, exist_ok=True)
    
    new_square = alter_square(square)
    # Write the square to local
    
    # Write the image with index to local
    cv.imwrite(f'{output_dir}/digit_{number_inside_square}.jpg', new_square)
    # Rotate and translate the square
    for angle in range(-6, 7, 2):
        M = cv.getRotationMatrix2D((new_square.shape[1] // 2, new_square.shape[0] // 2), angle, 1)
        rotated_square = cv.warpAffine(new_square, M, (new_square.shape[1], new_square.shape[0]))
        cv.imwrite(f'{output_dir}/digit_{number_inside_square}_rotated_{angle}.jpg', rotated_square)
        
        for dx in range(-6, 6, 2):
            for dy in range(-6, 6, 2):
                if dx == 0 and dy == 0:
                    continue
                translated_square = np.roll(rotated_square, shift=(dy, dx), axis=(0, 1))
                cv.imwrite(f'{output_dir}/digit_{number_inside_square}_rotated_{angle}_translated_{dx}_{dy}.jpg', translated_square)
    
    return square

# Incarcam imaginile din dataset
dataset = []
# Pentru fiecare imagine din dataset, incarcam imaginea si o adaugam in lista de imagini
dataset_folder = 'dataset'
for digit_folder in os.listdir(dataset_folder):
    digit_folder_path = os.path.join(dataset_folder, digit_folder)
    dir_list = os.listdir(digit_folder_path)
    for digit_image in dir_list:
        print("Reading image", digit_image)
        image_path = os.path.join(digit_folder_path, digit_image)
        image = cv.imread(image_path, cv.IMREAD_GRAYSCALE)
        # Get the digit from the folder name
        digit = int(digit_folder.split('_')[1])
        dataset.append((image, digit))
        
def classify_digit(digit_image):
    # Pentru fiecare imagine din dataset, calculam distanta euclidiana fata de imaginea de test
    print(f"Calculating distances for {len(dataset)} images")
    distances = []
    for image, digit in dataset:
        # Calculam distanta euclidiana
        distance = np.linalg.norm(digit_image - image)
        distances.append((distance, digit))
        
    # Sortam distantele
    distances.sort(key=lambda x: x[0])
    # Luam primele 5 distante
    top5 = distances[:5]
    # Calculam frecventa cifrelor
    digit_frequencies = {}
    for distance, digit in top5:
        if digit in digit_frequencies:
            digit_frequencies[digit] += 1
        else:
            digit_frequencies[digit] = 1
    # Sortam cifrele dupa frecventa
    sorted_frequencies = sorted(digit_frequencies.items(), key=lambda x: x[1], reverse=True)
    # Returnam cifra cu cea mai mare frecventa
    print(f"Litera detectata este {sorted_frequencies[0][0]}")
    return sorted_frequencies[0][0]


Reading image digit_13_rotated_2_translated_-6_-4.jpg
Reading image digit_13_rotated_-4_translated_2_0.jpg
Reading image digit_13_rotated_0_translated_2_-2.jpg
Reading image digit_13_rotated_4_translated_2_4.jpg
Reading image digit_13_rotated_2_translated_2_0.jpg
Reading image digit_13_rotated_-4_translated_0_-6.jpg
Reading image digit_13_rotated_-6_translated_0_-4.jpg
Reading image digit_13_rotated_2_translated_4_0.jpg
Reading image digit_13_rotated_-2_translated_-6_4.jpg
Reading image digit_13_rotated_6_translated_-4_-2.jpg
Reading image digit_13_rotated_-4_translated_-4_4.jpg
Reading image digit_13_rotated_-4_translated_2_-6.jpg
Reading image digit_13_rotated_-6_translated_0_-6.jpg
Reading image digit_13_rotated_2_translated_-4_2.jpg
Reading image digit_13_rotated_6_translated_2_0.jpg
Reading image digit_13_rotated_-4_translated_-6_-2.jpg
Reading image digit_13_rotated_6_translated_2_4.jpg
Reading image digit_13_rotated_2_translated_-6_2.jpg
Reading image digit_13_rotated_4_translat

In [22]:
def clasifica_fiecare_celula_din_imagine(image):
    image = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    masca_pentru_a_vedea_pozitiile_cifrelor_inf = np.array([0, 0, 95])
    masca_pentru_a_vedea_pozitiile_cifrelor_sup = np.array([255, 255, 255])
    # Aplicam masca pentru a vedea pozitiile cifrelor
    combined_mask = cv.inRange(image, masca_pentru_a_vedea_pozitiile_cifrelor_inf, masca_pentru_a_vedea_pozitiile_cifrelor_sup)
    # show_image('imagine_cu_careul_de_procesat',combined_mask)
    
    pozitii_deja_utilizate = []
    # Impartim imaginea in 14x14
    width, height = combined_mask.shape[:2]
    square_width, square_height = width // 14 + 1, height // 14 + 1
    # Pentru fiecare patrat, calculam media intensitatii
    for i in range(14):
        for j in range(14):
            # Calculam media intensitatii
            square = combined_mask[i * square_width:(i + 1) * square_width, j * square_height:(j + 1) * square_height]
            # Create a new square that is distance 20 away from the original square, inside the original square
            new_square = combined_mask[i * square_width + 20:(i + 1) * square_width - 20, j * square_height + 20:(j + 1) * square_height - 20]
            # Draw the new square referring to its shape
            # cvs.rectangle(combined_mask, (j * square_height + 20, i * square_width + 20), ((j + 1) * square_height - 20, (i + 1) * square_width - 20), (0, 255, 0), 2)
            
            # Calculam media intensitatii
            print(np.mean(new_square))
            mean_intensity = np.mean(new_square)
            # Daca media intensitatii este mai mica decat 100, atunci este un patrat negru
            if mean_intensity < 240 and mean_intensity > 0:
                # Create a new square that is distance 10 away from the original square, inside the original square
                new_square = combined_mask[i * square_width + 10:(i + 1) * square_width - 10, j * square_height + 10:(j + 1) * square_height - 10]
                # Draw the new square referring to its shape
                # cv.rectangle(combined_mask, (j * square_height + 10, i * square_width + 10), ((j + 1) * square_height - 10, (i + 1) * square_width - 10), (0, 255, 0), 2)
                inverted_square = 255 - new_square
                # Make all the pixels that are not 0, 255
                inverted_square[inverted_square > 0] = 255
                
                altered_square = alter_square(inverted_square)
                # Show the inverted square
                show_image('inverted_square',altered_square)
                
                cifra = classify_digit(altered_square)
                
                # Avem un patrat plin, il coloram cu grey
                combined_mask[i * square_width:(i + 1) * square_width, j * square_height:(j + 1) * square_height] = 100
                
                pozitii_deja_utilizate.append((mean_intensity, i, j, cifra))
    return pozitii_deja_utilizate, combined_mask
    

In [23]:

def clasifica_doar_o_celula(celula_din_imagine):
    image = cv.cvtColor(celula_din_imagine, cv.COLOR_BGR2HSV)
    masca_pentru_a_vedea_pozitiile_cifrelor_inf = np.array([0, 0, 95])
    masca_pentru_a_vedea_pozitiile_cifrelor_sup = np.array([255, 255, 255])
    # Aplicam masca pentru a vedea pozitiile cifrelor
    combined_mask = cv.inRange(image, masca_pentru_a_vedea_pozitiile_cifrelor_inf, masca_pentru_a_vedea_pozitiile_cifrelor_sup)
    
    square_width, square_height = combined_mask.shape
    new_square = combined_mask[10:square_width - 10, 10:square_height - 10]
    
    inverted_square = 255 - new_square
    # Make all the pixels that are not 0, 255
    inverted_square[inverted_square > 0] = 255
    
    # Print the inverted square
    # show_image('inverted_square',inverted_square)
    
    altered_square = alter_square(inverted_square)
    cifra = classify_digit(altered_square)
    return cifra

In [24]:

import os
lista_curenta_pozitii = {} # Pentru fiecare runda, retinem o lista cu pozitiile ocupate in acea runda
files = []
# Cream un folder curent, test
path_fisiere_antrenare = '../Proiect1/CAVA-2024-Tema1/antrenare'
path_fisiere_test = '../Proiect1/CAVA-2024-Tema1/evaluare/fake_test'

os.makedirs('test', exist_ok=True)
for imagine_path in os.listdir(path_fisiere_antrenare):
    # If it is a jpg file
    if not imagine_path.endswith('.jpg'):
        continue
    files.append(imagine_path.split('.')[0])

# Sortam fisierele
files.sort(key=lambda x: (int(x.split('_')[0]), int(x.split('_')[1])))

for imagine_path in files:
    # Cream key-ul pentru runda curenta
    if imagine_path.split('_')[0] not in lista_curenta_pozitii:
        lista_curenta_pozitii[imagine_path.split('_')[0]] = []
        
    print("Analizam imaginea", imagine_path)
    modified, orig = select_board_position_and_border_size(cv.imread(f'../Proiect1/CAVA-2024-Tema1/antrenare/{imagine_path}.jpg'))
    grid_intern_de_procesat = select_inner_grid_position_and_size_in_board(modified, orig)
    sqwidth, sqheight, pozitii_chr, pozitii_tuple = altereaza_imaginea_cu_grid_ul(grid_intern_de_procesat)
    print(pozitii_tuple)
    
    # Adaugam la lista de pozitii pozitiile curente, daca nu sunt deja adaugate
    for pozitie, (y, x) in zip(pozitii_chr, pozitii_tuple):
        if pozitie not in lista_curenta_pozitii[imagine_path.split('_')[0]]:
            lista_curenta_pozitii[imagine_path.split('_')[0]].append(pozitie)
            
            # Print the position and the x and y
            print(pozitie, x, y)
            
            # Get the square at the required position
            celula = grid_intern_de_procesat[y * sqwidth:(y + 1) * sqwidth, x * sqheight:(x + 1) * sqheight]
            
            # Show the cell
            # show_image('celula',celula)
            
            clasificare_celula = clasifica_doar_o_celula(celula)
            
            with open(f'test/{imagine_path}.txt', 'a') as f:
                f.write(pozitie + " " + f"{clasificare_celula}" + '\n')
            break


Analizam imaginea 1_01
square_width 143
square_height 143
[(8, 7)]
9H 7 8
Calculating distances for 11638 images
Litera detectata este 8
Analizam imaginea 1_02
square_width 143
square_height 143
[(8, 7), (9, 7)]
10H 7 9
Calculating distances for 11638 images
Litera detectata este 32
Analizam imaginea 1_03
square_width 143
square_height 143
[(8, 7), (9, 7), (10, 7)]
11H 7 10
Calculating distances for 11638 images
Litera detectata este 45
Analizam imaginea 1_04
square_width 143
square_height 143
[(8, 7), (9, 7), (10, 7), (11, 7)]
12H 7 11
Calculating distances for 11638 images
Litera detectata este 72
Analizam imaginea 1_05
square_width 143
square_height 143
[(5, 7), (8, 7), (9, 7), (10, 7), (11, 7)]
6H 7 5
Calculating distances for 11638 images
Litera detectata este 8
Analizam imaginea 1_06
square_width 143
square_height 143
[(4, 7), (5, 7), (8, 7), (9, 7), (10, 7), (11, 7)]
5H 7 4
Calculating distances for 11638 images
Litera detectata este 10
Analizam imaginea 1_07
square_width 143
sq